---
# **PyVHR using the CVP Dataset**
---

# Imports

In [ ]:
# -- MAIN IMPORT

import pyVHR as vhr
from pyVHR.BVP import *
from pyVHR.utils.errors import *
import numpy as np
import pandas as pd
import cv2
import os
from tqdm import tqdm

# Plotting: set 'colab' for Google Colaboratory, 'notebook' otherwise
vhr.plot.VisualizeParams.renderer = 'notebook'

# Function Definitions

## Dataframe Functions

### Create Instance DataFrame

In [ ]:
def create_instance_dataframe(results_path, base_path):
    """
    Create or load a DataFrame with instance-related information.

    Parameters:
        results_path (str): The path to save the DataFrame.
        base_path (str): The base path containing all video directories.
        
    Returns:
        pd.DataFrame: DataFrame containing instance-related information.
    """

    # Define the JSON filename for DataFrame storage
    filename = os.path.join(results_path, "instance_info.json")

    # Columns for the DataFrame
    columns = [ "INSTANCE",
                "DATAPATH", 
                "K1_PATH", 
                "K2_PATH", 
                "K1_PROCESSED", 
                "K2_PROCESSED", 
                "GT_PROCESSED", 
                "K1_SKIN_RECTANGLE_COORDS",
                "K1_PIXEL_THRESHOLD",
                "K1_SKIN_RECTANGLE_MEAN_RGB",
                "K2_SKIN_RECTANGLE_COORDS",
                "K2_PIXEL_THRESHOLD",
                "K2_SKIN_RECTANGLE_MEAN_RGB",
                "K1_PATCH_IDS",
                "K2_PATCH_IDS"]

    if os.path.exists(filename):
        # Load DataFrame if already exists
        df = pd.read_json(filename)
        print(f"DataFrame loaded from {filename}")
    else:
        # Initialize DataFrame with pre-defined columns
        df = pd.DataFrame(columns=columns)

        # A list to store unique instances
        unique_instances = []

        # List all directories in the base path
        all_dirs = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]

        for dir_name in all_dirs:
            instance, camera = dir_name.split("_")
            instance = int(instance)  # Convert to integer
            
            if instance not in unique_instances:
                unique_instances.append(instance)
                
                # Prepare a new DataFrame row
                new_row = pd.DataFrame([{
                    'INSTANCE': instance, 
                    'DATAPATH': None,
                    'K1_PATH': None,
                    'K2_PATH': None,
                    'K1_SKIN_RECTANGLE_COORDS': (-1,-1,-1,-1),
                    'K1_SKIN_RECTANGLE_MEAN_RGB': [-1,-1,-1],
                    'K1_PIXEL_THRESHOLD': -1,
                    'K2_SKIN_RECTANGLE_COORDS': (-1,-1,-1,-1),
                    'K2_SKIN_RECTANGLE_MEAN_RGB': [-1,-1,-1],
                    'K2_PIXEL_THRESHOLD': -1,
                    'K1_PATCH_NUMBERS': [],
                    'K2_PATCH_NUMBERS': [],
                }], columns=columns)
                
                # Append the new row to DataFrame using concat
                df = pd.concat([df, new_row], ignore_index=True)

            # Populate K1_PATH and K2_PATH fields
            shortened_path = os.path.join(base_path, dir_name) + "\\"
            if camera == 'K1':
                k1_path = shortened_path + "K1_Cropped_Colour.mkv"
                # Check if K1 file exists
                if os.path.exists(k1_path):
                    df.loc[df['INSTANCE'] == instance, 'K1_PATH'] = k1_path
                    df.loc[df['INSTANCE'] == instance, 'DATAPATH'] = shortened_path + "data.csv"
                else:
                    df.loc[df['INSTANCE'] == instance, 'K1_PATH'] = None

            elif camera == 'K2':
                k2_path = shortened_path + "K2_Cropped_Colour.mkv"                
                # Check if K2 file exists
                if os.path.exists(k2_path):
                    df.loc[df['INSTANCE'] == instance, 'K2_PATH'] = k2_path
                    df.loc[df['INSTANCE'] == instance, 'DATAPATH'] = shortened_path + "data.csv"
                else:
                    df.loc[df['INSTANCE'] == instance, 'K2_PATH'] = None

        # Sort DataFrame and reset index
        df.sort_values('INSTANCE', inplace=True)
        df.reset_index(drop=True, inplace=True)

        # Save DataFrame to JSON file
        df.to_json(filename)

    return df

### Create Groundtruth Dataframe

In [ ]:
def create_groundtruth_dataframe(instance_df, results_path):
    """
    Create a DataFrame named groundtruth_df, drawing the INSTANCE column from instance_df,
    and saves it as a JSON file.

    Parameters:
        instance_df (pd.DataFrame): The DataFrame containing instance-related information.
        results_path (str): The path to save the groundtruth_df as a JSON file.

    Returns:
        pd.DataFrame: DataFrame containing groundtruth-related information.
    """
    
    # Define the JSON filename for DataFrame storage
    filename = os.path.join(results_path, "groundtruth.json")
    
    if os.path.exists(filename):
        # Load DataFrame if already exists
        groundtruth_df = pd.read_json(filename)
        print(f"DataFrame loaded from {filename}")
    else:
        # Initialize DataFrame with data types and initial values
        columns = ["INSTANCE", "LENGTH", "SAMPLE_RATE", "ECG", "ECG_TIMES", 
                   "ECG_BPM", "ABP", "ABP_TIMES", "ABP_BPM", "CVP", 
                   "CVP_TIMES", "CVP_BPM"]

        initial_values = {
            "INSTANCE": [],
            "LENGTH": [],
            "SAMPLE_RATE": [],
            "ECG": [],
            "ECG_TIMES": [],
            "ECG_BPM": [],
            "ABP": [],
            "ABP_TIMES": [],
            "ABP_BPM": [],
            "CVP": [],
            "CVP_TIMES": [],
            "CVP_BPM": []
        }
        
        groundtruth_df = pd.DataFrame(initial_values)
        
        # Populate the INSTANCE column from instance_df
        groundtruth_df['INSTANCE'] = instance_df['INSTANCE'].astype(int).copy()
        
        # Fill the rest of the columns with default values
        for column in ["LENGTH"]:
            groundtruth_df[column] = -1  # Default value for int
        for column in ["SAMPLE_RATE"]:
            groundtruth_df[column] = -1.0  # Default value for float
        for column in ["ECG", "ECG_TIMES", "ECG_BPM", "ABP", "ABP_TIMES", "ABP_BPM", "CVP", "CVP_TIMES", "CVP_BPM"]:
            groundtruth_df[column] = groundtruth_df[column].apply(lambda x: [])  # Default value for lists

        # Save DataFrame to JSON file
        groundtruth_df.to_json(filename)
        print(f"DataFrame saved to {filename}")

    return groundtruth_df


### Create Holistic Dataframe

In [ ]:
def create_holistic_dataframes(methods, results_path):
    """
    Create a DataFrame for each method in the 'methods' list and save it as a JSON file.

    Parameters:
        methods (list): List of method names.
        results_path (str): The path to save the DataFrames as JSON files.
    """

    # Define the common columns for all DataFrames
    common_columns = ["INSTANCE", "TIMES", "GT"]

    # Define metrics for raw and segmented types
    raw_metrics = ["BPM_RAW", "RMSE_RAW", "MAE_RAW", "PCC_RAW", "CCC_RAW", "SNR_RAW"]
    segmented_metrics = ["BPM_SEGMENTED", "RMSE_SEGMENTED", "MAE_SEGMENTED", "PCC_SEGMENTED", "CCC_SEGMENTED", "SNR_SEGMENTED"]

    # Create K1 and K2 metrics by prefixing the raw and segmented metrics
    k1_metrics = [f"K1_{metric}" for metric in raw_metrics + segmented_metrics]
    k2_metrics = [f"K2_{metric}" for metric in raw_metrics + segmented_metrics]

    # Combine all columns
    all_columns = common_columns + k1_metrics + k2_metrics

    # Create a DataFrame for each method and save it
    for method in methods:
        # Create an empty DataFrame
        df_name = f"holistic_{method}"
        df = pd.DataFrame(columns=all_columns).astype('object')

        # Define JSON filename for storing the DataFrame
        filename = os.path.join(results_path, f"{df_name}.json")

        # Save DataFrame to JSON file
        df.to_json(filename)

### Create Patch Dataframe

In [ ]:
def create_patch_dataframes(methods, results_path):
    """
    Create a DataFrame for each method in the 'methods' list using the 'patch' prefix
    and save it as a JSON file.

    Parameters:
        methods (list): List of method names.
        results_path (str): The path to save the DataFrames as JSON files.
    """

    # Define the common columns for all DataFrames
    common_columns = ["INSTANCE", "TIMES","GT"]
    metrics = ["BPM", "RMSE", "MAE", "PCC", "CCC", "SNR"]

    # Generate columns for K1 and K2 with MED and PSD
    k1_med_columns = [f"K1_MED_{metric}" for metric in metrics]
    k1_psd_columns = [f"K1_PSD_{metric}" for metric in metrics]
    
    k2_med_columns = [f"K2_MED_{metric}" for metric in metrics]
    k2_psd_columns = [f"K2_PSD_{metric}" for metric in metrics]

    # Combine all columns
    all_columns = common_columns + k1_med_columns + k1_psd_columns + k2_med_columns + k2_psd_columns

    # Create a DataFrame for each method and save it
    for method in methods:
        # Create an empty DataFrame
        df_name = f"patch_{method}"
        df = pd.DataFrame(columns=all_columns).astype('object')

        # Define JSON filename for storing the DataFrame
        filename = os.path.join(results_path, f"{df_name}.json")

        # Save DataFrame to JSON file
        df.to_json(filename)

### Create Deep Holistic Dataframe

In [ ]:
def create_deep_holistic_dataframes(methods, results_path):
    """
    Create a DataFrame for each method in the 'methods' list using the 'deep_patch' prefix
    and save it as a JSON file.

    Parameters:
        methods (list): List of method names.
        results_path (str): The path to save the DataFrames as JSON files.
    """

    # Define the common columns for all DataFrames
    common_columns = ["INSTANCE", "TIMES", "GT"]

    # Define metrics for raw and segmented types
    raw_metrics = ["BPM_RAW", "RMSE_RAW", "MAE_RAW", "PCC_RAW", "CCC_RAW", "SNR_RAW"]
    segmented_metrics = ["BPM_SEGMENTED", "RMSE_SEGMENTED", "MAE_SEGMENTED", "PCC_SEGMENTED", "CCC_SEGMENTED", "SNR_SEGMENTED"]

    # Create K1 and K2 metrics by prefixing the raw and segmented metrics
    k1_metrics = [f"K1_{metric}" for metric in raw_metrics + segmented_metrics]
    k2_metrics = [f"K2_{metric}" for metric in raw_metrics + segmented_metrics]

    # Combine all columns
    all_columns = common_columns + k1_metrics + k2_metrics

    # Create a DataFrame for each method and save it
    for method in methods:
        # Create an empty DataFrame
        df_name = f"deep_holistic_{method}"
        df = pd.DataFrame(columns=all_columns).astype('object')

        # Define JSON filename for storing the DataFrame
        filename = os.path.join(results_path, f"{df_name}.json")

        # Save DataFrame to JSON file
        df.to_json(filename)


### Create Deep Patch Dataframe

In [ ]:
def create_deep_patch_dataframes(methods, results_path):
    """
    Create a DataFrame for each method in the 'methods' list using the 'deep_holistic' prefix
    and save it as a JSON file. This version of the function also accounts for MED and PSD estimations
    for both RAW and SEGMENTED data types.

    Parameters:
        methods (list): List of method names.
        results_path (str): The path to save the DataFrames as JSON files.
    """

    # Define the common columns for all DataFrames
    common_columns = ["INSTANCE", "TIMES", "GT"]
    metrics = ["BPM", "RMSE", "MAE", "PCC", "CCC", "SNR"]
    
    # Create estimation types and data types
    estimation_types = ["MED", "PSD"]
    data_types = ["RAW", "SEGMENTED"]

    # Create columns for K1 and K2 with MED, PSD, RAW, and SEGMENTED
    k1_columns = [f"K1_{est}_{dtype}_{metric}" for est in estimation_types for dtype in data_types for metric in metrics]
    k2_columns = [f"K2_{est}_{dtype}_{metric}" for est in estimation_types for dtype in data_types for metric in metrics]

    # Combine all columns
    all_columns = common_columns + k1_columns + k2_columns

    # Create a DataFrame for each method and save it
    for method in methods:
        # Create an empty DataFrame
        df_name = f"deep_patch_{method}"
        df = pd.DataFrame(columns=all_columns).astype('object')

        # Define JSON filename for storing the DataFrame
        filename = os.path.join(results_path, f"{df_name}.json")

        # Save DataFrame to JSON file
        df.to_json(filename)

## Groundtruth Functions

### Process Groundtruth

In [ ]:
def read_ground_truth_data(instance_df, groundtruth_df, results_path, instance, dataset, window_info=[8, 1]):
    """
    Read the ground truth data for ECG, ABP, and CVP signals.
    
    Parameters:
    - instance_df (pd.DataFrame): DataFrame containing the instance paths.
    - groundtruth_df (pd.DataFrame): DataFrame to store the BPM and time data.
    - instance (str): The instance identifier to locate the row in the DataFrame.
    - dataset: An object containing methods for reading signal files.
    - window_info (list): [window size, step] for BPM calculation.
    
    Returns:
    - pd.DataFrame: DataFrame with updated BPM and time data.
    """

    instance_filename = os.path.join(results_path, "instance_info.json")
    groundtruth_filename = os.path.join(results_path, "groundtruth.json")
    
    # Get the index of the instance
    index = instance_df[instance_df['INSTANCE'] == int(instance)].index[0]
        
    if groundtruth_df.at[index, 'LENGTH'] != -1:
        print(f"Instance has already been processed. Skipping.")
        return instance_df, groundtruth_df

    try:
        sigFileName = instance_df.loc[instance_df['INSTANCE'] == int(instance), 'DATAPATH'].values[0]
    except Exception as e:
        print(f"Error in locating DATAPATH: {e}")
        sys.exit(1)

    wsize = window_info[0]

    def read_signal(signal_type):
        try:
            sigGT = dataset.readSigfile(sigFileName, signalGT=signal_type)
            
            # Get the raw signal
            raw_signal = sigGT.data

            # Get the BPM and time data
            bpmGT, timesGT = sigGT.getBPM(wsize)
            
            idx = groundtruth_df[groundtruth_df['INSTANCE'] == int(instance)].index[0]
            
            groundtruth_df.at[idx, signal_type] = raw_signal
            groundtruth_df.at[idx, f"{signal_type}_TIMES"] = timesGT
            groundtruth_df.at[idx, f"{signal_type}_BPM"] = bpmGT
            groundtruth_df.at[idx, "LENGTH"] = len(raw_signal[0])
            groundtruth_df.at[idx, "SAMPLE_RATE"] = dataset.SIG_SampleRate
            return groundtruth_df
            
        except Exception as e:
            return groundtruth_df

    # Reading ECG, ABP, and CVP signals
    for signal_type in ["ECG", "ABP", "CVP"]:
        groundtruth_df = read_signal(signal_type)

    instance_df.to_json(instance_filename)
    groundtruth_df.to_json(groundtruth_filename)

    return instance_df, groundtruth_df


### Populate Groundtruth Dataframe

In [ ]:
def populate_groundtruth_df(instance_df, results_path, dataset, window_info=[8, 1]):

    # Populate the ground truth dataframe
    groundtruth_df = create_groundtruth_dataframe(instance_df, results_path)
    instances = instance_df['INSTANCE'].to_list()
    for instance in tqdm(instances,desc="Processing instances"):
        try:
            instance_df, groundtruth_df = read_ground_truth_data(instance_df, groundtruth_df, results_path, instance, dataset, window_info)
        except Exception as e:
            continue

    return instance_df, groundtruth_df

## Initialise sig_extractor Functions

In [ ]:
def initialize_sig_extractor(seconds=0,patch_info = [40,0]):
    patch_size = patch_info[0]
    overlap = patch_info[1]
    sig_extractor = vhr.extraction.SignalProcessing()
    sig_extractor.set_visualize_skin_and_landmarks( visualize_skin=True,
                                                    visualize_landmarks=True, 
                                                    visualize_landmarks_number=False, 
                                                    visualize_patch=True)
    sig_extractor.choose_cuda_device(0)
    sig_extractor.set_skin_extractor(vhr.extraction.SkinExtractionRectangle('GPU')) # Set the skin extractor
    sig_extractor.set_square_patches_side(patch_info[0])
    sig_extractor.set_overlap(overlap)
    sig_extractor.thickness = 1
    sig_extractor.font_size = 0.3
    return sig_extractor

## Patch Processing Functions

In [ ]:
def add_fixed_patch_info(fixed_patches,patch_ids,patch_bvps,patch_bpmES,timesES):
    for fixed_patch in fixed_patches:
        for window in range(len(patch_ids)):
            # loop through each patch, check if it's the one we're after, and append if it is, including the 
            for patch in range(len(patch_ids[window])):
                if patch_ids[window][patch] == fixed_patch.ID:
                    fixed_patch.times.append(timesES[window])
                    fixed_patch.bvps.append(np.array([patch_bvps[window][patch]]))
                    fixed_patch.bpms.append(np.array(patch_bpmES[window][patch]))
    return fixed_patches

In [ ]:
def update_patch_snrs(fixed_patches,gt_bpms,fps):
    for fixed_patch in fixed_patches:
        if len(fixed_patch.times) > 0:
            fixed_patch.snr = []
            for i, bvp in enumerate(fixed_patch.bvps):
                fixed_patch.snr.append(get_SNR([bvp],fps,gt_bpms,[fixed_patch.times[i]]))
            fixed_patch.snr = np.array(fixed_patch.snr)
    return fixed_patches

In [ ]:
def update_patch_errors(fixed_patches,gt_times,gt_bpms):
    for fixed_patch in fixed_patches:
        if len(fixed_patch.times) > 0:
            temp_bpms = np.expand_dims(np.array(fixed_patch.bpms),axis=0)
            fixed_patch.rmse = RMSEerror(temp_bpms,gt_bpms,fixed_patch.times,gt_times)
            fixed_patch.mae = MAEerror(temp_bpms,gt_bpms,fixed_patch.times,gt_times)
            fixed_patch.max = MAXError(temp_bpms,gt_bpms,fixed_patch.times,gt_times)
    return fixed_patches

In [ ]:
def update_patch_metrics(fixed_patches,gt_times,gt_bpms,fps):
    try:
        fixed_patches = update_patch_snrs(fixed_patches,gt_bpms,fps)
    except Exception as e:
        print(f"Error in calculating SNR: {e}")
    try:
        fixed_patches = update_patch_errors(fixed_patches,gt_times,gt_bpms)
    except Exception as e:
        print(f"Error in calculating errors: {e}")
    return fixed_patches

In [ ]:
def make_patch_dataframe(fixed_patches):    
    patch_data = []
    for fixed_patch in fixed_patches:
        patch_data.append([fixed_patch.ID,
                        fixed_patch.x_min,
                        fixed_patch.x_max,
                        fixed_patch.y_min,
                        fixed_patch.y_max,
                        np.array(fixed_patch.times),
                        np.array(fixed_patch.bvps),
                        np.array(fixed_patch.bpms),
                        fixed_patch.rmse,
                        fixed_patch.mae,
                        fixed_patch.max,
                        np.array(fixed_patch.snr)])
    df = pd.DataFrame(patch_data,columns = ["ID","x_min","x_max","y_min","y_max","times","bvps","bpms","RMSE","MAE","MAX","SNRs"])
    return df

## Skin Segmentation Functions

In [ ]:
def get_skin_segmentation_info(instance_df,results_path,skin_threshold=30):

    # Define the list of the instances
    instance_filename = os.path.join(results_path, "instance_info.json")

    # Define the list of the instances
    instances = instance_df['INSTANCE'].to_list()

    # Set up the sig_extractor
    sig_extractor = vhr.extraction.SignalProcessing()
    sig_extractor.choose_cuda_device(0)
    sig_extractor.set_skin_extractor(vhr.extraction.SkinExtractionRectangle('GPU')) # Set the skin extractor


    # Loop through all the instances, open the k1 and k2 video (if they exist)
    # get the rectangle coordinates and the mean RGB values, add them to the 
    # instance_df dataframe
    for instance in tqdm(instances,desc="Processing instances"):
        # Define the paths for saving the frames
        frames_path = os.path.join(results_path, "figures", str(instance))
        k1_display_frame_path = os.path.join(frames_path, "K1_display_frames.png")
        k2_display_frame_path = os.path.join(frames_path, "K2_display_frames.png")
        k1_skin_frame_path = os.path.join(frames_path, "K1_skin_frames.png")
        k2_skin_frame_path = os.path.join(frames_path, "K2_skin_frames.png")

        # Get the index of the instance
        index_to_update = instance_df[instance_df['INSTANCE'] == int(instance)].index[0]

        # Process the K1 video if we can
        if instance_df.at[index_to_update, 'K1_SKIN_RECTANGLE_COORDS'] == [-1, -1, -1, -1]:
            # Check if the K1 video exists
            if instance_df.loc[instance_df['INSTANCE'] == int(instance), 'K1_PATH'].values[0] is None:
                continue
            else:
                # Reset the sig_extractor
                sig_extractor.skin_extractor.mean_rgb = None
                sig_extractor.skin_extractor.rect = None
                sig_extractor.display_frame = None
                sig_extractor.skin_frame = None
                # Get the video path for the K1
                videoFilename = instance_df.loc[instance_df['INSTANCE'] == int(instance), 'K1_PATH'].values[0]
                # Get the values
                hol_sig = sig_extractor.extract_holistic_rectangle(videoFilename,skin_threshold)
                # Add the values to the dataframe
                instance_df.at[index_to_update, 'K1_SKIN_RECTANGLE_COORDS'] = tuple(sig_extractor.skin_extractor.rect)
                instance_df.at[index_to_update, 'K1_SKIN_RECTANGLE_MEAN_RGB'] = sig_extractor.skin_extractor.mean_rgb.tolist()
                instance_df.at[index_to_update, 'K1_PIXEL_THRESHOLD'] = skin_threshold
                # Save the frames
                cv2.imwrite(k1_display_frame_path, sig_extractor.display_frame)
                cv2.imwrite(k1_skin_frame_path, sig_extractor.display_skin_frame)
                # Save the instance dataframe
                instance_df.to_json(instance_filename)

        
        # Process the K2 video if we can
        if instance_df.at[index_to_update, 'K2_SKIN_RECTANGLE_COORDS'] == [-1, -1, -1, -1]:
            # Check if the K2 video exists
            if instance_df.loc[instance_df['INSTANCE'] == int(instance), 'K2_PATH'].values[0] is None:
                continue
            else:
                # Reset the sig_extractor
                sig_extractor.skin_extractor.mean_rgb = None
                sig_extractor.skin_extractor.rect = None
                sig_extractor.display_frame = None
                sig_extractor.skin_frame = None
                # Get the video path for the K2
                videoFilename = instance_df.loc[instance_df['INSTANCE'] == int(instance), 'K2_PATH'].values[0]
                # Get the values
                hol_sig = sig_extractor.extract_holistic_rectangle(videoFilename,skin_threshold)
                # Add the values to the dataframe
                instance_df.at[index_to_update, 'K2_SKIN_RECTANGLE_COORDS'] = tuple(sig_extractor.skin_extractor.rect)
                instance_df.at[index_to_update, 'K2_SKIN_RECTANGLE_MEAN_RGB'] = sig_extractor.skin_extractor.mean_rgb.tolist()
                instance_df.at[index_to_update, 'K2_PIXEL_THRESHOLD'] = skin_threshold
                # Save the frames
                cv2.imwrite(k2_display_frame_path, sig_extractor.display_frame)
                cv2.imwrite(k2_skin_frame_path, sig_extractor.display_skin_frame)
                # Save the instance dataframe
                instance_df.to_json(instance_filename)

    # read in the saved instance dataframe
    instance_df = pd.read_json(instance_filename)
            
    return instance_df

## Process Holistic Function

In [ ]:
def holistic_process_video(instance,
                            camera,
                            instance_df, 
                            groundtruth_df, 
                            results_path, 
                            sig_extractor, 
                            methods,
                            deep_methods, 
                            window_info, 
                            seconds, 
                            pixel_thresholds, 
                            frequency_bandpass):
    """ 
    Runs the pipeline on a specific video file.

    Args:
        instance (int):
            - The specific instance we want to analyze
        camera (str):
            - Either "K1" or "K2", specifies the camera we want to use 
        instance_df (pd.DataFrame):
            - The instance dataframe containing paths to the videos and other information
        groundtruth_df (pd.DataFrame):
            - The groundtruth dataframe containing the BPM and time data
        results_path (str):
            - The path to save the results
        sig_extractor (pyVHR.extraction.SignalProcessing):
            - The signal extractor object
        methods (list):
            - The list of methods to use for the traditional processing
        deep_methods (list):
            - The list of methods to use for the deep processing
        window_info (list):
            - The window size and stride for the traditional processing
        seconds (int):
            - The number of seconds to process
        pixel_thresholds (list): 
            - The pixel thresholds for the traditional processing
        frequency_bandpass (list):
            - The frequency bandpass for the traditional processing
    """

    # Set up the variables
    wsize = window_info[0]
    stride = window_info[1]
    low_thr = pixel_thresholds[0]
    high_thr = pixel_thresholds[1]
    low_hz = frequency_bandpass[0]
    high_hz = frequency_bandpass[1]

    # Get the index of the instance
    index = instance_df[instance_df['INSTANCE'] == int(instance)].index[0]

    # Load the relevant information from the instance_df
    videoFilename = instance_df.at[index, camera + '_PATH']
    skin_threshold = instance_df.at[index, camera + '_PIXEL_THRESHOLD']
    mean_rgb = instance_df.at[index, camera + '_SKIN_RECTANGLE_MEAN_RGB']

    # Update the sig_extractor based on this video
    fps = vhr.extraction.get_fps(videoFilename)
    sig_extractor.set_total_frames(fps*seconds)
    sig_extractor.skin_extractor.mean_rgb = mean_rgb

    # Try load the groundtruth data from the groundtruth_df
    # Signal types to check
    signal_types = ['ECG', 'ABP', 'CVP']

    # Initialize variables
    bpmGT = None
    timesGT = None
    signal_type = None

    # Try to load the ground truth data
    try:
        for signal_type in signal_types:
            bpm_values = groundtruth_df.at[index, f'{signal_type}_BPM']
            times_values = groundtruth_df.at[index, f'{signal_type}_TIMES']
            
            if not np.isnan(bpm_values[0]):
                bpmGT = bpm_values
                timesGT = times_values
                signal_type = signal_type
                break
    except Exception as e:
        print(f"Error in loading ground truth data: {e}")

    if signal_type is None:
        print("No groundtruth data found for this instance.")
        return instance_df, groundtruth_df, sig_extractor

    # Get the holistic signal for the raw frames
    try:
        hol_sig = sig_extractor.extract_holistic_rectangle(videoFilename,skin_threshold,segmented_frames = False)
        windowed_hol_sig, timesES = vhr.extraction.sig_windowing(hol_sig, wsize, 1, fps)
        filtered_windowed_hol_sig = vhr.BVP.apply_filter(windowed_hol_sig, vhr.BVP.rgb_filter_th, fps=fps, params={'RGB_LOW_TH': low_thr, 'RGB_HIGH_TH': high_thr})
        filtered_windowed_hol_sig = vhr.BVP.apply_filter(filtered_windowed_hol_sig, vhr.BVP.BPfilter, params={'order':6,'minHz':low_hz,'maxHz':high_hz,'fps':fps})
        for method in methods:
            results_df = pd.read_json(os.path.join(results_path, f"holistic_{method}.json"))
            if method == "CHROM":
                hol_bvps = RGB_sig_to_BVP(filtered_windowed_hol_sig, fps, device_type='cuda', method=cupy_CHROM)
            elif method == "LGI":
                hol_bvps = RGB_sig_to_BVP(filtered_windowed_hol_sig, fps, device_type='cpu', method=cpu_LGI)
            elif method == "POS":
                hol_bvps = RGB_sig_to_BVP(filtered_windowed_hol_sig, fps, device_type='cuda', method=cupy_POS, params={'fps':fps})
            elif method == "PBV":
                hol_bvps = RGB_sig_to_BVP(filtered_windowed_hol_sig, fps, device_type='cpu', method=cpu_PBV)
            elif method == "PCA":
                hol_bvps = RGB_sig_to_BVP(filtered_windowed_hol_sig, fps, device_type='cpu', method=cpu_PCA, params={'component':'all_comp'})
            elif method == "GREEN":
                hol_bvps = RGB_sig_to_BVP(filtered_windowed_hol_sig, fps, device_type='cpu', method=cpu_GREEN)
            elif method == "OMIT":
                hol_bvps = RGB_sig_to_BVP(filtered_windowed_hol_sig, fps, device_type='cpu', method=cpu_OMIT)
            elif method == "ICA":
                hol_bvps = RGB_sig_to_BVP(filtered_windowed_hol_sig, fps, device_type='cpu', method=cpu_ICA, params={'component':'all_comp'})
            else:
                print(f"Method {method} not recognized. Skipping.")
                break
            hol_bvps = vhr.BVP.apply_filter(hol_bvps, BPfilter, params={'order':6,'minHz':low_hz,'maxHz':high_hz,'fps':fps})
            hol_bpmES = vhr.BPM.BVP_to_BPM_cuda(hol_bvps, fps)
            if method == "PCA":
                hol_bpmES = [x[0] for x in hol_bpmES]
            if method == "ICA":
                hol_bpmES = [x[0] for x in hol_bpmES]
            RMSE, MAE, MAX, PCC, CCC, SNR = getErrors(hol_bvps, fps, hol_bpmES, bpmGT, timesES, timesGT)
            # Update the results_dataframes
            results_df.at[index, 'INSTANCE'] = instance
            results_df.at[index, 'GT'] = signal_type
            results_df['TIMES'] = results_df['TIMES'].astype('object')
            results_df.at[index, 'TIMES'] = timesES
            results_df[camera+'_BPM_RAW'] = results_df[camera+'_BPM_RAW'].astype('object')
            hol_bpmES = [x.item() for x in hol_bpmES]
            results_df.at[index, camera+'_BPM_RAW'] = hol_bpmES
            results_df.at[index, camera+'_RMSE_RAW'] = RMSE.tolist()
            results_df.at[index, camera+'_MAE_RAW'] = MAE.tolist()
            results_df.at[index, camera+'_PCC_RAW'] = PCC.tolist()
            results_df.at[index, camera+'_CCC_RAW'] = CCC.tolist()
            results_df.at[index, camera+'_SNR_RAW'] = SNR.tolist()

            # Save the results_df
            results_df = results_df.convert_dtypes()
            results_df.to_json(os.path.join(results_path, f"holistic_{method}.json"))

    except Exception as e:
        print(f"Error in processing the raw holistic signal: {e}")

    # Get the holistic signal for the raw frames
    hol_sig = sig_extractor.extract_holistic_rectangle(videoFilename,skin_threshold,segmented_frames = True)
    windowed_hol_sig, timesES = vhr.extraction.sig_windowing(hol_sig, wsize, 1, fps)
    filtered_windowed_hol_sig = vhr.BVP.apply_filter(windowed_hol_sig, vhr.BVP.rgb_filter_th, fps=fps, params={'RGB_LOW_TH': low_thr, 'RGB_HIGH_TH': high_thr})
    filtered_windowed_hol_sig = vhr.BVP.apply_filter(filtered_windowed_hol_sig, vhr.BVP.BPfilter, params={'order':6,'minHz':low_hz,'maxHz':high_hz,'fps':fps})
    for method in methods:
        results_df = pd.read_json(os.path.join(results_path, f"holistic_{method}.json"))
        if method == "CHROM":
            hol_bvps = RGB_sig_to_BVP(filtered_windowed_hol_sig, fps, device_type='cuda', method=cupy_CHROM)
        elif method == "LGI":
            hol_bvps = RGB_sig_to_BVP(filtered_windowed_hol_sig, fps, device_type='cpu', method=cpu_LGI)
        elif method == "POS":
            hol_bvps = RGB_sig_to_BVP(filtered_windowed_hol_sig, fps, device_type='cuda', method=cupy_POS, params={'fps':fps})
        elif method == "PBV":
            hol_bvps = RGB_sig_to_BVP(filtered_windowed_hol_sig, fps, device_type='cpu', method=cpu_PBV)
        elif method == "PCA":
            hol_bvps = RGB_sig_to_BVP(filtered_windowed_hol_sig, fps, device_type='cpu', method=cpu_PCA, params={'component':'all_comp'})
        elif method == "GREEN":
            hol_bvps = RGB_sig_to_BVP(filtered_windowed_hol_sig, fps, device_type='cpu', method=cpu_GREEN)
        elif method == "OMIT":
            hol_bvps = RGB_sig_to_BVP(filtered_windowed_hol_sig, fps, device_type='cpu', method=cpu_OMIT)
        elif method == "ICA":
            hol_bvps = RGB_sig_to_BVP(filtered_windowed_hol_sig, fps, device_type='cpu', method=cpu_ICA, params={'component':'all_comp'})
        else:
            print(f"Method {method} not recognized. Skipping.")
            break
        hol_bvps = vhr.BVP.apply_filter(hol_bvps, BPfilter, params={'order':6,'minHz':low_hz,'maxHz':high_hz,'fps':fps})
        hol_bpmES = vhr.BPM.BVP_to_BPM_cuda(hol_bvps, fps)
        if method == "PCA":
            hol_bpmES = [x[0] for x in hol_bpmES]
        if method == "ICA":
            hol_bpmES = [x[0] for x in hol_bpmES]
        RMSE, MAE, MAX, PCC, CCC, SNR = getErrors(hol_bvps, fps, hol_bpmES, bpmGT, timesES, timesGT)
        # Update the results_dataframes
        results_df.at[index, 'INSTANCE'] = instance
        results_df.at[index, 'GT'] = signal_type
        results_df['TIMES'] = results_df['TIMES'].astype('object')
        results_df.at[index, 'TIMES'] = timesES
        results_df[camera+'_BPM_SEGMENTED'] = results_df[camera+'_BPM_SEGMENTED'].astype('object')
        hol_bpmES = [x.item() for x in hol_bpmES]
        results_df.at[index, camera+'_BPM_SEGMENTED'] = hol_bpmES
        results_df.at[index, camera+'_RMSE_SEGMENTED'] = RMSE.tolist()
        results_df.at[index, camera+'_MAE_SEGMENTED'] = MAE.tolist()
        results_df.at[index, camera+'_PCC_SEGMENTED'] = PCC.tolist()
        results_df.at[index, camera+'_CCC_SEGMENTED'] = CCC.tolist()
        results_df.at[index, camera+'_SNR_SEGMENTED'] = SNR.tolist()

        # Save the results_df
        results_df = results_df.convert_dtypes()
        results_df.to_json(os.path.join(results_path, f"holistic_{method}.json"))

    ############ Deep Processing ############

    # Get the raw and segmented frames for the deep learning models
    frames = sig_extractor.extract_raw(videoFilename)
    #print(frames.shape)
    skin_frames = np.array(sig_extractor.visualize_skin_collection)
    #print(skin_frames.shape)

    for method in deep_methods:
        print(f"Processing {method} - Deep Holistic")
        deep_results_df = pd.read_json(os.path.join(results_path, f"deep_holistic_{method}.json"))
        deep_results_df.at[index, 'INSTANCE'] = instance
        deep_results_df.at[index, 'GT'] = signal_type
        deep_results_df['TIMES'] = deep_results_df['TIMES'].astype('object')
        deep_results_df[camera+'_BPM_RAW'] = deep_results_df[camera+'_BPM_RAW'].astype('object')
        deep_results_df[camera+'_BPM_SEGMENTED'] = deep_results_df[camera+'_BPM_SEGMENTED'].astype('object')

        if method == 'MTTS_CAN':
            # first do the raw frames
            bvp_pred = vhr.deepRPPG.MTTS_CAN_deep(frames, fps, verb=0)
            bvps = vhr.BPM.BVPsignal(bvp_pred, fps) # BVP object
            bvp_win, timesES = BVP_windowing(bvp_pred, wsize, fps, stride=1)
            bpmES = vhr.BPM.BVP_to_BPM_cuda(bvp_win, fps) 
            RMSE, MAE, MAX, PCC, CCC, SNR = vhr.utils.getErrors(bvp_win, fps, bpmES, bpmGT, timesES, timesGT)
            deep_results_df.at[index, 'TIMES'] = timesES
            deep_results_df.at[index, camera+'_BPM_RAW'] = [x.item() for x in bpmES]
            deep_results_df.at[index, camera+'_RMSE_RAW'] = RMSE.tolist()
            deep_results_df.at[index, camera+'_MAE_RAW'] = MAE.tolist()
            deep_results_df.at[index, camera+'_PCC_RAW'] = PCC.tolist()
            deep_results_df.at[index, camera+'_CCC_RAW'] = CCC.tolist()
            deep_results_df.at[index, camera+'_SNR_RAW'] = SNR.tolist()
            # Now do the segmented frames
            bvp_pred = vhr.deepRPPG.MTTS_CAN_deep(skin_frames, fps, verb=0)
            bvps = vhr.BPM.BVPsignal(bvp_pred, fps) # BVP object
            bvp_win, timesES = BVP_windowing(bvp_pred, wsize, fps, stride=1)
            bpmES = vhr.BPM.BVP_to_BPM_cuda(bvp_win, fps)
            RMSE, MAE, MAX, PCC, CCC, SNR = vhr.utils.getErrors(bvp_win, fps, bpmES, bpmGT, timesES, timesGT)
            deep_results_df.at[index, camera+'_BPM_SEGMENTED'] = [x.item() for x in bpmES]
            deep_results_df.at[index, camera+'_RMSE_SEGMENTED'] = RMSE.tolist()
            deep_results_df.at[index, camera+'_MAE_SEGMENTED'] = MAE.tolist()
            deep_results_df.at[index, camera+'_PCC_SEGMENTED'] = PCC.tolist()
            deep_results_df.at[index, camera+'_CCC_SEGMENTED'] = CCC.tolist()
            deep_results_df.at[index, camera+'_SNR_SEGMENTED'] = SNR.tolist()
            # Save the results_df
            deep_results_df = deep_results_df.convert_dtypes()
            deep_results_df.to_json(os.path.join(results_path, f"deep_holistic_{method}.json"))
        elif method == 'HR_CNN':
            # apply HR_CNN model
            bvp_pred = vhr.deepRPPG.HR_CNN_bvp_pred(frames)
            bvps = vhr.BPM.BVPsignal(bvp_pred, fps) # raw BVP object, mostly to set up sample rate
            bvp_win, timesES = BVP_windowing(bvp_pred, wsize, fps, stride=1)
            bpmES = vhr.BPM.BVP_to_BPM_cuda(bvp_win, fps)
            RMSE, MAE, MAX, PCC, CCC, SNR = vhr.utils.getErrors(bvp_win, fps, bpmES, bpmGT, timesES, timesGT)
            deep_results_df.at[index, camera+'_BPM_RAW'] = [x.item() for x in bpmES]
            deep_results_df.at[index, camera+'_RMSE_RAW'] = RMSE.tolist()
            deep_results_df.at[index, camera+'_MAE_RAW'] = MAE.tolist()
            deep_results_df.at[index, camera+'_PCC_RAW'] = PCC.tolist()
            deep_results_df.at[index, camera+'_CCC_RAW'] = CCC.tolist()
            deep_results_df.at[index, camera+'_SNR_RAW'] = SNR.tolist()
            # Now do the segmented frames
            bvp_pred = vhr.deepRPPG.HR_CNN_bvp_pred(skin_frames)
            bvps = vhr.BPM.BVPsignal(bvp_pred, fps) # BVP object
            bvp_win, timesES = BVP_windowing(bvp_pred, wsize, fps, stride=1)
            bpmES = vhr.BPM.BVP_to_BPM_cuda(bvp_win, fps)
            RMSE, MAE, MAX, PCC, CCC, SNR = vhr.utils.getErrors(bvp_win, fps, bpmES, bpmGT, timesES, timesGT)
            deep_results_df.at[index, camera+'_BPM_SEGMENTED'] = [x.item() for x in bpmES]
            deep_results_df.at[index, camera+'_RMSE_SEGMENTED'] = RMSE.tolist()
            deep_results_df.at[index, camera+'_MAE_SEGMENTED'] = MAE.tolist()
            deep_results_df.at[index, camera+'_PCC_SEGMENTED'] = PCC.tolist()
            deep_results_df.at[index, camera+'_CCC_SEGMENTED'] = CCC.tolist()
            deep_results_df.at[index, camera+'_SNR_SEGMENTED'] = SNR.tolist()
            # Save the results_df
            deep_results_df = deep_results_df.convert_dtypes()
            deep_results_df.to_json(os.path.join(results_path, f"deep_holistic_{method}.json"))
        else:
            print("Method not found")
            continue

    return instance_df, groundtruth_df, sig_extractor

## Process Patch

In [ ]:
def patch_process_instance(instance,
                            camera,
                            instance_df, 
                            groundtruth_df, 
                            results_path, 
                            sig_extractor, 
                            methods,
                            deep_methods, 
                            window_info, 
                            seconds,
                            pixel_thresholds, 
                            frequency_bandpass):
    # Patch process for a single instance, with a bunch of methods
    # Return the instance_df, groundtruth_df and the sig_extractor. 

    # Create a sub_folder for the instance
    figure_dir = os.path.join(results_path, "figures")
    instance_figure_dir = os.path.join(figure_dir, f"{instance}")
    os.makedirs(instance_figure_dir, exist_ok=True)


    # Save the display_image and the skin_image to the instance_figure_dir
    # convert from BGR to RGB first
    display_image = cv2.cvtColor(sig_extractor.display_frame, cv2.COLOR_BGR2RGB)
    skin_image = cv2.cvtColor(sig_extractor.display_skin_frame, cv2.COLOR_BGR2RGB)
    cv2.imwrite(os.path.join(instance_figure_dir, f"{camera}_display_image.png"), display_image)
    cv2.imwrite(os.path.join(instance_figure_dir, f"{camera}_skin_image.png"), skin_image)

    # Set up the variables
    wsize = window_info[0]
    stride = window_info[1]
    low_thr = pixel_thresholds[0]
    high_thr = pixel_thresholds[1]
    low_hz = frequency_bandpass[0]
    high_hz = frequency_bandpass[1]
    patch_shape = (int(sig_extractor.square),int(sig_extractor.square))
    overlap = sig_extractor.overlap

    # Get the index of the instance
    index = instance_df[instance_df['INSTANCE'] == int(instance)].index[0]

    # Try load the groundtruth data from the groundtruth_df
    # Signal types to check
    signal_types = ['ECG', 'ABP', 'CVP']

    # Initialize variables
    bpmGT = None
    timesGT = None
    signal_type = None

    # Try to load the ground truth data
    for s_type in signal_types:
        bpm_values = groundtruth_df.at[index, f'{s_type}_BPM']
        times_values = groundtruth_df.at[index, f'{s_type}_TIMES']
        
        if not np.isnan(bpm_values[0]):
            bpmGT = bpm_values
            timesGT = times_values
            signal_type = s_type
            break

    if signal_type is None:
        print("No groundtruth data found for this instance.")
        return instance_df, groundtruth_df, sig_extractor

    # Get relevant info from the instance_df
    videoFilename = instance_df.at[index, camera + '_PATH']

    # Update the sig extractor
    fps = vhr.extraction.get_fps(videoFilename)
    sig_extractor.set_total_frames(fps*seconds)
    sig_extractor.set_fixed_patches(videoFilename,region_type="squares",overlap=overlap)
    sig_extraction_method = "mean"

    # Get the patch signal
    try:
        patch_sig = sig_extractor.extract_fixed_patches(sig_extraction_method,segmented_frames = True)
    except Exception as e:
        print(f"Error in extracting the patch signal {instance} {camera}: {e}")
    # Window the patch_signal
    try:
        windowed_patch_sig, timesES = vhr.extraction.sig_windowing(patch_sig, wsize, stride, fps)
    except Exception as e:
        print(f"Error in windowing the patch signal {instance} {camera}: {e}")
    # Pixel threshold the signal
    try:
        filtered_windowed_patch_sig, patch_ids = vhr.BVP.apply_custom_filter(windowed_patch_sig, vhr.BVP.rgb_filter_th_with_ids, params={'RGB_LOW_TH': low_thr, 'RGB_HIGH_TH': high_thr})
    except Exception as e:
        print(f"Error in pixel thresholding the patch signal {instance} {camera}: {e}")
    # Bandpass the signal
    try:
        filtered_windowed_patch_sig = vhr.BVP.apply_filter(filtered_windowed_patch_sig, vhr.BVP.BPfilter, params={'order':6,'minHz':low_hz,'maxHz':high_hz,'fps':fps})
    except Exception as e:
        print(f"Error in bandpassing the patch signal {instance} {camera}: {e}")

    # Update the instance_df
    try:
        instance_df[camera+'_PATCH_IDS'] = instance_df[camera+'_PATCH_IDS'].astype('object')
        instance_df.at[index, camera+'_PATCH_IDS'] = [array.tolist() for array in patch_ids]
    except Exception as e:
        print(f"Error in updating the instance_df {instance} {camera}: {e}")

    # Loop through the methods
    for method in methods:
        try:
            results_df = pd.read_json(os.path.join(results_path, f"patch_{method}.json"))
        except Exception as e:
            print(f"Error in reading the results_df for {instance} {camera} {method}: {e}")
            continue
        # Update the first few columns of the dataframe
        try:
            results_df.at[index, 'INSTANCE'] = instance
            results_df.at[index, 'GT'] = signal_type
            results_df['TIMES'] = results_df['TIMES'].astype('object')
            results_df.at[index, 'TIMES'] = timesES
        except Exception as e:
            print(f"Error in updating the results_df for  {instance} {camera} {method}: {e}")
            continue

        if method == "CHROM":
            try:
                patch_bvps = RGB_sig_to_BVP(filtered_windowed_patch_sig, fps, device_type='cuda', method=cupy_CHROM)
            except Exception as e:
                print(f"Error in processing the patch signal with  {instance} {camera} {method}: {e}")
                continue
        elif method == "LGI":
            try:
                patch_bvps = RGB_sig_to_BVP(filtered_windowed_patch_sig, fps, device_type='cpu', method=cpu_LGI)
            except Exception as e:
                print(f"Error in processing the patch signal with  {instance} {camera} {method}: {e}")
                continue
        elif method == "POS":
            try:
                patch_bvps = RGB_sig_to_BVP(filtered_windowed_patch_sig, fps, device_type='cuda', method=cupy_POS, params={'fps':fps})
            except Exception as e:
                print(f"Error in processing the patch signal with  {instance} {camera} {method}: {e}")
                continue
        elif method == "PBV":
            try:
                patch_bvps = RGB_sig_to_BVP(filtered_windowed_patch_sig, fps, device_type='cpu', method=cpu_PBV)
            except Exception as e:
                print(f"Error in processing the patch signal with  {instance} {camera} {method}: {e}")
                continue
        elif method == "PCA":
            try:
                patch_bvps = RGB_sig_to_BVP(filtered_windowed_patch_sig, fps, device_type='cpu', method=cpu_PCA, params={'component':'all_comp'})
            except Exception as e:
                print(f"Error in processing the patch signal with  {instance} {camera} {method}: {e}")
                continue
        elif method == "GREEN":
            try:
                patch_bvps = RGB_sig_to_BVP(filtered_windowed_patch_sig, fps, device_type='cpu', method=cpu_GREEN)
            except Exception as e:
                print(f"Error in processing the patch signal with  {instance} {camera} {method}: {e}")
                continue
        elif method == "OMIT":
            try:
                patch_bvps = RGB_sig_to_BVP(filtered_windowed_patch_sig, fps, device_type='cpu', method=cpu_OMIT)
            except Exception as e:
                print(f"Error in processing the patch signal with  {instance} {camera} {method}: {e}")
                continue
        elif method == "ICA":
            try:
                patch_bvps = RGB_sig_to_BVP(filtered_windowed_patch_sig, fps, device_type='cpu', method=cpu_ICA, params={'component':'all_comp'})
            except Exception as e:
                print(f"Error in processing the patch signal with  {instance} {camera} {method}: {e}")
                continue
        else:
            print("Method not found")
            continue

        # Post processing bandpass filter
        try:
            patch_bvps = vhr.BVP.apply_filter(patch_bvps, BPfilter, params={'order':6,'minHz':low_hz,'maxHz':high_hz,'fps':fps})
        except Exception as e:
            print(f"Error in post processing the patch signal with  {instance} {camera} {method}: {e}")
            continue
        
        # Get the bpm estimates for all the patches
        try:
            patch_bpmES = vhr.BPM.BVP_to_BPM_cuda(patch_bvps, fps)
        except Exception as e:
            print(f"Error in getting the bpm estimates for  {instance} {camera} {method}: {e}")
            continue
        
        try:
            sig_extractor.fixed_patches = add_fixed_patch_info(sig_extractor.fixed_patches,patch_ids,patch_bvps,patch_bpmES,timesES)
        except Exception as e:
            print(f"Error in adding the patch info to sig_extractor {instance} {camera} {method}: {e}")
            continue

        try:
            sig_extractor.fixed_patches = update_patch_metrics(sig_extractor.fixed_patches,timesGT,bpmGT,fps)
        except Exception as e:
            print(f"Error in updating the patch metrics {instance} {camera} {method}: {e}")

        # make the patch dataframe which we will then save
        try:
            patch_df = make_patch_dataframe(sig_extractor.fixed_patches)
        except Exception as e:
            print(f"Error in making the patch dataframe for instance {instance} {camera} {method}: {e}")
            continue

        try:
            patch_df_dir = os.path.join(results_path, "patch_dataframes", str(instance))
            os.makedirs(patch_df_dir, exist_ok=True)
            patch_df.to_json(os.path.join(patch_df_dir, f"{method}_{camera}_patch_df.json"))
        except Exception as e:
            print(f"Error in saving the patch dataframe at instance {instance} {camera} {method}: {e}")
            continue

        try:
            for fixed_patch in sig_extractor.fixed_patches:
                fixed_patch.times = []
                fixed_patch.bvps = []
                fixed_patch.bpms = []
                fixed_patch.snrs = []
                fixed_patch.rmse = 0
                fixed_patch.mae = 0
                fixed_patch.max = 0
        except Exception as e:
            print(f"Error in resetting the fixed patches  of sig_extractor {instance} {camera} {method}: {e}")
            continue


        bpm_estimation_methods = ["MED", "PSD"]

        for est_method in bpm_estimation_methods:
            if est_method == "MED":
                bpmES, MAD = vhr.BPM.BPM_median(patch_bpmES)

            if est_method == "PSD":
                ma = vhr.extraction.MotionAnalysis(sig_extractor, wsize, fps)
                bpmES = vhr.BPM.BPM_clustering(ma, patch_bvps, fps, wsize, movement_thrs=None, opt_factor=0.5)

            try:
                RMSE, MAE, MAX, PCC, CCC, SNR = getErrors(patch_bvps, fps, bpmES, bpmGT, timesES, timesGT)
                results_df[camera+'_'+est_method+'_BPM'] = results_df[camera+'_'+est_method+'_BPM'].astype('object')
                bpmES = [x.item() for x in bpmES]
            except Exception as e:
                print(f"Error in getting the errors for {instance} {camera} method {method} and est_method {est_method}: {e}")
                continue
            
            try:
                results_df.at[index, camera+'_'+est_method+'_BPM'] = bpmES
                results_df.at[index, camera+'_'+est_method+'_RMSE'] = RMSE.tolist()
                results_df.at[index, camera+'_'+est_method+'_MAE'] = MAE.tolist()
                results_df.at[index, camera+'_'+est_method+'_PCC'] = PCC.tolist()
                results_df.at[index, camera+'_'+est_method+'_CCC'] = CCC.tolist()
                results_df.at[index, camera+'_'+est_method+'_SNR'] = SNR.tolist()
            except Exception as e:
                print(f"Error in updating the results_df for {instance} {camera} method {method} and est_method {est_method}: {e}")
                continue

            try:
                # Save the results_df
                results_df.to_json(os.path.join(results_path, f"patch_{method}.json"))
                print(f'successfully saved the results_df for camera {camera}, method {method} and est_method {est_method}')
            except Exception as e:
                print(f"Error in saving the results_df for instance {instance} {camera} {method} and est_method {est_method}: {e}")
                continue

            """            
            # Make the folders for saving some figures
            try:
                estimated_fig_dir = os.path.join(instance_figure_dir, f"{method}_{est_method}_{camera}_estimated")
                error_fig_dir = os.path.join(instance_figure_dir, f"{method}_{est_method}_{camera}_error")
                os.makedirs(estimated_fig_dir, exist_ok=True)
                os.makedirs(error_fig_dir, exist_ok=True)
            except Exception as e:
                print(f"Error in making the figure directories for method {method} and est_method {est_method}: {e}")
                continue

            # Get the figures
            image = sig_extractor.display_frame
            try:
                for wind in range(len(patch_bvps)):
                    ldmks, fig1 = vhr.plot.visualize_BVPs_heatmap(image, patch_bvps, patch_ids, wind, patch_shape, overlap, fps, minHz=0.65, maxHz=4)
                    fig2 = vhr.plot.visualize_BPM_Errors_heatmap(image, ldmks, timesES, wind, timesGT, bpmGT, patch_shape, overlap,vmin=-20,vmax=20)
                    fig1_path = os.path.join(estimated_fig_dir, f"{wind}.png")
                    fig2_path = os.path.join(error_fig_dir, f"{wind}.png")
                    cv2.imwrite(fig1_path, fig1)
                    cv2.imwrite(fig2_path, fig2)
            except Exception as e:
                print(f"Error in getting the figures for method {method} and est_method {est_method}: {e}")
                continue
            """

    # save the instance_df
    try:
        instance_df.to_json(os.path.join(results_path, "instance_info.json"))
    except Exception as e:
        print(f"Error in saving the instance_df: {e}")
        return instance_df, groundtruth_df, sig_extractor


    return instance_df, groundtruth_df, sig_extractor

## User Inputs

In [ ]:
## User input parameters
dataset_name = 'CVP'
window_info = [8,1]     # window size and stride
seconds = 0     # seconds of video to be processed (0 for all video)
skin_threshold = 30    # threshold for skin extraction
pixel_thresholds = [75,230] #low and high thresholding
frequency_bandpass = [0.65,4.0] # bandpass range
patch_info = [10.,0] # size, pixel overlap

vhr.extraction.SkinProcessingParams.RGB_LOW_TH =  pixel_thresholds[0]
vhr.extraction.SkinProcessingParams.RGB_HIGH_TH = pixel_thresholds[1]

vhr.extraction.SignalProcessingParams.RGB_LOW_TH = pixel_thresholds[0]
vhr.extraction.SignalProcessingParams.RGB_HIGH_TH = pixel_thresholds[1]

### Create the dataframes

In [ ]:
results_path = "C:\\Users\\20759193\\source\\repos\\pyVHR\\results"
base_path = "C:\\Users\\20759193\\source\\repos\\pyVHR\\data"
dataset_name = 'CVP'
methods = ['CHROM','LGI','POS','PBV','PCA','GREEN','OMIT','ICA','PCA']
deep_methods = ['HR_CNN','MTTS_CAN']
dataset = vhr.datasets.datasetFactory(dataset_name, videodataDIR="", BVPdataDIR="")
instance_df = create_instance_dataframe(results_path, base_path)
groundtruth_df = create_groundtruth_dataframe(instance_df, results_path)
create_holistic_dataframes(methods, results_path)
create_patch_dataframes(methods, results_path)
create_deep_holistic_dataframes(deep_methods, results_path)
#create_deep_patch_dataframes(deep_methods, results_path)

### Populate the groundtruth df

In [ ]:
instance_df, groundtruth_df = populate_groundtruth_df(instance_df, results_path, dataset, window_info=window_info)

### Segment the skin for all the frames

In [ ]:
instance_df = get_skin_segmentation_info(instance_df,results_path,skin_threshold)

# Test instance

In [ ]:
instances = instance_df['INSTANCE'].values
cameras = ['K1','K2']
# loop through them using tqdm, with a description
for instance in tqdm(instances[0:1], desc="Processing instance:"):
    for camera in cameras:
        try:
            sig_extractor = initialize_sig_extractor(seconds=seconds,patch_info = patch_info)
            instance_df, groundtruth_df, sig_extractor = holistic_process_video(instance,
                                    camera,
                                    instance_df, 
                                    groundtruth_df, 
                                    results_path, 
                                    sig_extractor, 
                                    methods,
                                    deep_methods, 
                                    window_info, 
                                    seconds, 
                                    pixel_thresholds, 
                                    frequency_bandpass)
        except:
            print(f"Error processing holistic instance {instance} for camera {camera}")
            continue
        
        try:
            instance_df, groundtruth_df, sig_extractor = patch_process_instance(instance,
                                    camera,
                                    instance_df, 
                                    groundtruth_df, 
                                    results_path, 
                                    sig_extractor, 
                                    methods,
                                    deep_methods, 
                                    window_info, 
                                    seconds, 
                                    pixel_thresholds, 
                                    frequency_bandpass)
        except:
            print(f"Error processing patch instance {instance} for camera {camera}")
            continue       
        

In [ ]:
df = make_patch_dataframe(sig_extractor.fixed_patches)

In [ ]:
df.head()

## Process Videos

### Function for single video processing patches

### Running the processing functions

In [ ]:
dataset_name = 'CVP'
window_info = [8,1]     # window size and stride
seconds = 0     # seconds of video to be processed (0 for all video)
skin_threshold = 40    # threshold for skin extraction
pixel_thresholds = [5,250] #low and high thresholding
frequency_bandpass = [0.75,200/60] # bandpass range
patch_info = [40.,30] # size, pixel overlap
methods = ['CHROM', 'LGI', 'POS', 'PBV', 'GREEN', 'OMIT','ICA','PCA']
instance = 100
camera = "K1"

sig_extractor = initialize_sig_extractor(seconds=seconds,patch_info=patch_info)

print("Processing Holistic")
instance_df,groundtruth_df,sig_extractor = holistic_process_instance(instance,
                            camera,
                            instance_df, 
                            groundtruth_df, 
                            results_path, 
                            sig_extractor, 
                            methods, 
                            window_info, 
                            seconds, 
                            skin_threshold, 
                            pixel_thresholds, 
                            frequency_bandpass)
print("Holistic finished: Processing Patch")
instance_df,groundtruth_df,sig_extractor = patch_process_instance(instance,
                            camera,
                            instance_df, 
                            groundtruth_df, 
                            results_path, 
                            sig_extractor, 
                            methods, 
                            window_info, 
                            seconds, 
                            skin_threshold, 
                            pixel_thresholds, 
                            frequency_bandpass)

In [ ]:
method = "PCA"
results_df = pd.read_json(os.path.join(results_path, f"patch_{method}.json"))
results_df.head()

## Another thing

In [ ]:
def holistic_processing(instances_df,groundtruth_df,sig_extractor,methods,skin_threshold = 30,window_info = [8,1],pixel_thresholds = [5,230],frequency_bandpass = [0.65,4]):
    """
    Performs Holistic processing on a video to obtain BVP signals using various methods.
    
    Parameters:
    - Various parameters as required by internal processing
    """
    wsize = window_info[0]
    stride = window_info[1]
    skin_threshold = skin_threshold
    low_thr = pixel_thresholds[0]
    high_thr = pixel_thresholds[1]
    low_hz = frequency_bandpass[0]
    high_hz = frequency_bandpass[1]


    # Set the paths
    path, videoFilename, sigFilename = get_paths(df,instance,camera)

    # Load the best groundtruth we have
    bpmGT,timesGT = get_groundtruth(df, instance, camera)

    # Get fps
    fps = vhr.extraction.get_fps(videoFilename)

    # Extract the skin
    hol_sig = sig_extractor.extract_holistic_rectangle(videoFilename,skin_threshold)
    
    # Window the signal
    windowed_hol_sig, timesES = vhr.extraction.sig_windowing(hol_sig, wsize, 1, fps)

    # Preprocess 
    # Pixel Thresholding
    filtered_windowed_hol_sig = vhr.BVP.apply_filter(windowed_hol_sig, vhr.BVP.rgb_filter_th, fps=fps, params={'RGB_LOW_TH': low_thr, 'RGB_HIGH_TH': high_thr})
    
    # Bandpass
    filtered_windowed_hol_sig = vhr.BVP.apply_filter(filtered_windowed_hol_sig, vhr.BVP.BPfilter, params={'order':6,'minHz':low_hz,'maxHz':high_hz,'fps':fps})

    for method in methods:
        if method == "CHROM":
            print("Processing CHROM - Holistic")
            hol_bvps = RGB_sig_to_BVP(filtered_windowed_hol_sig, fps, device_type='cuda', method=cupy_CHROM)
        elif method == "LGI":
            print("PROCESSING LGI - Holistic")
            hol_bvps = RGB_sig_to_BVP(filtered_windowed_hol_sig, fps, device_type='cpu', method=cpu_LGI)
        elif method == "POS":
            print("PROCESSING POS - Holistic")
            hol_bvps = RGB_sig_to_BVP(filtered_windowed_hol_sig, fps, device_type='cuda', method=cupy_POS, params={'fps':fps})
        elif method == "PBV":
            print("PROCESSING PBV - Holistic")
            hol_bvps = RGB_sig_to_BVP(filtered_windowed_hol_sig, fps, device_type='cpu', method=cpu_PBV)
        elif method == "PCA":
            print("PROCESSING PCA - Holistic")
            hol_bvps = RGB_sig_to_BVP(filtered_windowed_hol_sig, fps, device_type='cpu', method=cpu_PCA, params={'component':'all_comp'})
        elif method == "GREEN":
            print("PROCESSING GREEN - Holistic")
            hol_bvps = RGB_sig_to_BVP(filtered_windowed_hol_sig, fps, device_type='cpu', method=cpu_GREEN)
        elif method == "OMIT":
            print("PROCESSING OMIT - Holistic")
            hol_bvps = RGB_sig_to_BVP(filtered_windowed_hol_sig, fps, device_type='cpu', method=cpu_OMIT)
        elif method == "ICA":
            print("PROCESSING ICA - Holistic")
            hol_bvps = RGB_sig_to_BVP(filtered_windowed_hol_sig, fps, device_type='cpu', method=cpu_ICA, params={'component':'all_comp'})
        else:
            print("Method not found")
            continue

        # Post-processing signals
        # Bandpass
        hol_bvps = vhr.BVP.apply_filter(hol_bvps, BPfilter, params={'order':6,'minHz':low_hz,'maxHz':high_hz,'fps':fps})

        # Get the BPM Estimates
        hol_bpmES = vhr.BPM.BVP_to_BPM_cuda(hol_bvps, fps)
        if method == "PCA":
            hol_bpmES = [x[0] for x in hol_bpmES]
        if method == "ICA":
            hol_bpmES = [x[0] for x in hol_bpmES]

        # Holistic errors
        RMSE, MAE, MAX, PCC, CCC, SNR = getErrors(hol_bvps, fps, hol_bpmES, bpmGT, timesES, timesGT)

        # Store the results
        ERRORS = [RMSE, MAE, MAX, PCC, CCC, SNR]
        OUTPUT = [timesES, hol_bpmES, ERRORS]
        df.at[df[df['INSTANCE'] == int(instance)].index[0], 'HOL_' + method + '_' + camera] = OUTPUT
    
    return df, sig_extractor


In [ ]:
def deep_processing(df,instance,camera,sig_extractor,window_info = [8,1]):

    # Set parameters
    wsize = window_info[0]
    stride = window_info[1]

    # Load the best groundtruth we have
    bpmGT,timesGT = get_groundtruth(df, instance, camera)
    path, videoFilename, sigFilename = get_paths(df,instance,camera)

    # Get the two different holistics to process
    fps = vhr.extraction.get_fps(videoFilename)
    raw_frames = sig_extractor.extract_raw(videoFilename)
    skin_frames = np.array(sig_extractor.visualize_skin_collection)

####################   MTTS_CAN   ####################
    # Predict bvp for raw_frames
    bvp_pred = vhr.deepRPPG.MTTS_CAN_deep(raw_frames, fps, verb=1)
    # Window bvp
    bvp_win, timesES = BVP_windowing(bvp_pred, wsize, fps, stride=stride)
    # Estimates
    bpmES = vhr.BPM.BVP_to_BPM_cuda(bvp_win, fps)
    # Errors
    RMSE, MAE, MAX, PCC, CCC, SNR = vhr.utils.getErrors(bvp_win, fps, bpmES, bpmGT, timesES, timesGT)
    # Store the results
    ERRORS = [RMSE, MAE, MAX, PCC, CCC, SNR]
    OUTPUT = [timesES, bpmES, ERRORS]
    df.at[df[df['INSTANCE'] == int(instance)].index[0], 'DEEP_MTTS_CAN_FULL_' + camera] = OUTPUT

    # Segmented Frames
    bvp_pred = vhr.deepRPPG.MTTS_CAN_deep(skin_frames, fps, verb=1)
    # Window bvp
    bvp_win, timesES = BVP_windowing(bvp_pred, wsize, fps, stride=stride)
    # Estimates
    bpmES = vhr.BPM.BVP_to_BPM_cuda(bvp_win, fps)
    # Errors
    RMSE, MAE, MAX, PCC, CCC, SNR = vhr.utils.getErrors(bvp_win, fps, bpmES, bpmGT, timesES, timesGT)
    # Store the results
    ERRORS = [RMSE, MAE, MAX, PCC, CCC, SNR]
    OUTPUT = [timesES, bpmES, ERRORS]
    df.at[df[df['INSTANCE'] == int(instance)].index[0], 'DEEP_MTTS_CAN_SEGMENTED_' + camera] = OUTPUT


####################   HR_CNN   ####################
    # Predict bvp for raw_frames
    bvp_pred = vhr.deepRPPG.HR_CNN_bvp_pred(raw_frames)
    # Window bvp
    bvp_win, timesES = BVP_windowing(bvp_pred, wsize, fps, stride=stride)
    # Estimates
    bpmES = vhr.BPM.BVP_to_BPM_cuda(bvp_win, fps)
    # Errors
    RMSE, MAE, MAX, PCC, CCC, SNR = vhr.utils.getErrors(bvp_win, fps, bpmES, bpmGT, timesES, timesGT)
    # Store results
    ERRORS = [RMSE, MAE, MAX, PCC, CCC, SNR]
    OUTPUT = [timesES, bpmES, ERRORS]
    df.at[df[df['INSTANCE'] == int(instance)].index[0], 'DEEP_HR_CNN_FULL_' + camera] = OUTPUT

    # Segmented Frames
    bvp_pred = vhr.deepRPPG.HR_CNN_bvp_pred(skin_frames)
    # Window bvp
    bvp_win, timesES = BVP_windowing(bvp_pred, wsize, fps, stride=stride)
    # Estimates
    bpmES = vhr.BPM.BVP_to_BPM_cuda(bvp_win, fps)
    # Errors
    RMSE, MAE, MAX, PCC, CCC, SNR = vhr.utils.getErrors(bvp_win, fps, bpmES, bpmGT, timesES, timesGT)
    # Store the results
    ERRORS = [RMSE, MAE, MAX, PCC, CCC, SNR]
    OUTPUT = [timesES, bpmES, ERRORS]
    df.at[df[df['INSTANCE'] == int(instance)].index[0], 'DEEP_HR_CNN_SEGMENTED_' + camera] = OUTPUT

    return df

In [ ]:
def patch_processing(sig_extractor, vhr, videoFileName, wsize, fps, methods):
    """
    Performs Patch processing on a video to obtain BVP signals using various methods.
    
    Parameters:
    - Various parameters as required by internal processing
    """
    # Set landmarks and extraction method
    sig_extractor.set_square_patches_side(40.)
    sig_extractor.set_fixed_landmarks(videoFileName, region_type="squares", overlap=20)
    patch_sig = sig_extractor.extract_fixed_patches("mean")
    
    # Window and filter the signal
    windowed_patch_sig, _ = vhr.extraction.sig_windowing(patch_sig, wsize, 1, fps)
    filtered_windowed_patch_sig, _ = vhr.BVP.apply_custom_filter(windowed_patch_sig, vhr.BVP.rgb_filter_th_with_ids, params={'RGB_LOW_TH': 5, 'RGB_HIGH_TH': 230})
    
    patch_bvps = {}  # Initialize an empty dictionary to store BVP signals
    
    for method in methods:
        print(f"PROCESSING {method} - Patches")
        
        # Device type and method mapping
        device_type = 'cuda' if method in ['CHROM', 'POS'] else 'cpu'
        
        # Process and store the BVP signal
        patch_bvps[method] = RGB_sig_to_BVP(filtered_windowed_patch_sig, fps, device_type=device_type, method=method)
    
    return patch_bvps

In [ ]:
def process_video(df,instance,camera):
    # Set the parameters
    seconds = 0     # seconds of video to be processed (0 for all video)
    pixel_threshold = 30
    wsize = 8
    dataset_name = 'CVP'

    # Initialise everything
    path = df.loc[df['INSTANCE'] == instance, camera + '_DATA_PATH'].values[0]
    videoFileName = path + camera + "_Cropped_Colour.mkv"
    fps = vhr.extraction.get_fps(videoFileName)
    sigFileName = path + "data.csv"
    dataset = vhr.datasets.datasetFactory(dataset_name, videodataDIR="", BVPdataDIR="")
    sig_extractor = vhr.extraction.SignalProcessing()   # Set the class
    sig_extractor.set_visualize_skin_and_landmarks(
      visualize_skin=True, 
      visualize_landmarks=True, 
      visualize_landmarks_number=True, 
      visualize_patch=True)
    sig_extractor.choose_cuda_device(0)                 # Set the GPU
    sig_extractor.set_skin_extractor(vhr.extraction.SkinExtractionRectangle('GPU')) # Set the skin extractor
    sig_extractor.set_total_frames(seconds*fps) # Set the number of frames
    

    # Initialise Parameters
    vhr.extraction.SkinProcessingParams.RGB_LOW_TH =  5     # threshold for skin extraction
    vhr.extraction.SkinProcessingParams.RGB_HIGH_TH = 230   # threshold for skin extraction
    vhr.extraction.SignalProcessingParams.RGB_LOW_TH = 5    # threshold for signal extraction
    vhr.extraction.SignalProcessingParams.RGB_HIGH_TH = 230 # threshold for signal extraction
    sig_extractor.set_visualize_skin_and_landmarks(visualize_skin=True, visualize_landmarks=True, visualize_landmarks_number=True, visualize_patch=True)

    # Get the ground truth data
    try:
        sigGT_ECG = dataset.readSigfile(sigFileName, signalGT='ECG')
        sigGT_ECG.show_ECG = True
        bpmGT_ECG, timesGT_ECG = sigGT_ECG.getBPM(wsize)
        # Add these to the DataFrame
        ecg_bpms = bpmGT_ECG.tolist()
        ecg_times = timesGT_ECG.tolist()
        ECG = [ecg_times, ecg_bpms]
        df.at[df[df['INSTANCE'] == int(instance)].index[0], 'ECG'] = ECG
    except Exception as e:
        print("ECG not found. Error:", e)
        sigGT_ECG = bpmGT_ECG = timesGT_ECG = None

    try:
        sigGT_ABP = dataset.readSigfile(sigFileName, signalGT='ABP')
        bpmGT_ABP, timesGT_ABP = sigGT_ABP.getBPM(wsize)
        # Add these to the DataFrame
        abp_bpms = bpmGT_ABP.tolist()
        abp_times = timesGT_ABP.tolist()
        ABP = [abp_times, abp_bpms]
        df.at[df[df['INSTANCE'] == int(instance)].index[0], 'ABP'] = ABP
    except Exception as e:
        print("ABP not found. Error:", e)
        sigGT_ABP = bpmGT_ABP = timesGT_ABP = None

    try:
        sigGT_CVP = dataset.readSigfile(sigFileName, signalGT='CVP')
        bpmGT_CVP, timesGT_CVP = sigGT_CVP.getBPM(wsize)
        # Add these to the DataFrame
        cvp_bpms = bpmGT_CVP.tolist()
        cvp_times = timesGT_CVP.tolist()
        CVP = [cvp_times, cvp_bpms]
        df.at[df[df['INSTANCE'] == int(instance)].index[0], 'CVP'] = CVP
    except Exception as e:
        print("CVP not found. Error:", e)
        sigGT_CVP = bpmGT_CVP = timesGT_CVP = None

    # For the patches, define our landmarks
    sig_extractor.set_square_patches_side(40.)
    sig_extractor.set_fixed_landmarks(videoFileName,region_type="squares",overlap=20)
    sig_extraction_method = "mean"

    # Prefilter before we use the specific method
    hol_sig = sig_extractor.extract_holistic_rectangle(videoFileName,pixel_threshold)    # Extract the signal
    patch_sig = sig_extractor.extract_fixed_patches(sig_extraction_method)

    # Save the 10th frame just to check filtering.
    frames = sig_extractor.extract_raw(videoFileName)
    print(frames.shape)
    skin_frames = np.array(sig_extractor.visualize_skin_collection)
    print(skin_frames.shape)

    cv2.imwrite(path + camera +'frame.png', cv2.cvtColor(frames[9], cv2.COLOR_RGB2BGR))
    cv2.imwrite(path +  camera +'skin_frame.png', cv2.cvtColor(skin_frames[9], cv2.COLOR_RGB2BGR))

    # Holistic Processing
    windowed_hol_sig, timesES = vhr.extraction.sig_windowing(hol_sig, wsize, 1, fps) # Window the signal
    filtered_windowed_hol_sig = vhr.BVP.apply_filter(windowed_hol_sig, vhr.BVP.rgb_filter_th, fps=fps, params={'RGB_LOW_TH': 5, 'RGB_HIGH_TH': 230}) # Apply the threshold filter
    filtered_windowed_hol_sig = vhr.BVP.apply_filter(filtered_windowed_hol_sig, vhr.BVP.BPfilter, params={'order':6,'minHz':0.65,'maxHz':4.0,'fps':fps}) # Apply the other filter

    # Patch Processing
    
    windowed_patch_sig, timesES = vhr.extraction.sig_windowing(patch_sig, wsize, 1, fps)
    filtered_windowed_patch_sig, patch_ids = vhr.BVP.apply_custom_filter(windowed_patch_sig, vhr.BVP.rgb_filter_th_with_ids, params={'RGB_LOW_TH': 5, 'RGB_HIGH_TH': 230})
    filtered_windowed_patch_sig = vhr.BVP.apply_filter(filtered_windowed_patch_sig, vhr.BVP.BPfilter, params={'order':6,'minHz':0.65,'maxHz':4.0,'fps':fps})

    methods = ['CHROM', 'LGI', 'POS', 'PBV', 'GREEN', 'OMIT','ICA','PCA']
    for method in methods:
        if method == "CHROM":
            print("Processing CHROM - Holistic")
            hol_bvps = RGB_sig_to_BVP(filtered_windowed_hol_sig, fps, device_type='cuda', method=cupy_CHROM)
            print("PROCESSING CHROM - Patches")
            patch_bvps = RGB_sig_to_BVP(filtered_windowed_patch_sig, fps, device_type='cuda', method=cupy_CHROM)
        elif method == "LGI":
            print("PROCESSING LGI - Holistic")
            hol_bvps = RGB_sig_to_BVP(filtered_windowed_hol_sig, fps, device_type='cpu', method=cpu_LGI)
            print("Processing LGI - Patches")
            patch_bvps = RGB_sig_to_BVP(filtered_windowed_patch_sig, fps, device_type='cpu', method=cpu_LGI)
        elif method == "POS":
            print("PROCESSING POS - Holistic")
            hol_bvps = RGB_sig_to_BVP(filtered_windowed_hol_sig, fps, device_type='cuda', method=cupy_POS, params={'fps':fps})
            print("PROCESSING POS - Patches")
            patch_bvps = RGB_sig_to_BVP(filtered_windowed_patch_sig, fps, device_type='cuda', method=cupy_POS, params={'fps':fps})
        elif method == "PBV":
            print("PROCESSING PBV - Holistic")
            hol_bvps = RGB_sig_to_BVP(filtered_windowed_hol_sig, fps, device_type='cpu', method=cpu_PBV)
            print("PROCESSING PBV - Patches")
            patch_bvps = RGB_sig_to_BVP(filtered_windowed_patch_sig, fps, device_type='cpu', method=cpu_PBV)
        elif method == "PCA":
            print("PROCESSING PCA - Holistic")
            hol_bvps = RGB_sig_to_BVP(filtered_windowed_hol_sig, fps, device_type='cpu', method=cpu_PCA, params={'component':'all_comp'})
            print("PROCESSING PCA - Patches")
            patch_bvps = RGB_sig_to_BVP(filtered_windowed_patch_sig, fps, device_type='cpu', method=cpu_PCA, params={'component':'all_comp'})
        elif method == "GREEN":
            print("PROCESSING GREEN - Holistic")
            hol_bvps = RGB_sig_to_BVP(filtered_windowed_hol_sig, fps, device_type='cpu', method=cpu_GREEN)
            print("PROCESSING GREEN - Patches")
            patch_bvps = RGB_sig_to_BVP(filtered_windowed_patch_sig, fps, device_type='cpu', method=cpu_GREEN)
        elif method == "OMIT":
            print("PROCESSING OMIT - Holistic")
            hol_bvps = RGB_sig_to_BVP(filtered_windowed_hol_sig, fps, device_type='cpu', method=cpu_OMIT)
            print("PROCESSING OMIT - Patches")
            patch_bvps = RGB_sig_to_BVP(filtered_windowed_patch_sig, fps, device_type='cpu', method=cpu_OMIT)
        elif method == "ICA":
            print("PROCESSING ICA - Holistic")
            hol_bvps = RGB_sig_to_BVP(filtered_windowed_hol_sig, fps, device_type='cpu', method=cpu_ICA, params={'component':'all_comp'})
            print("PROCESSING ICA - Patches")
            patch_bvps = RGB_sig_to_BVP(filtered_windowed_patch_sig, fps, device_type='cpu', method=cpu_ICA, params={'component':'all_comp'})
        else:
            print("Method not found")
            continue

        # Apply the filter            
        hol_bvps = vhr.BVP.apply_filter(hol_bvps, BPfilter, params={'order':6,'minHz':0.65,'maxHz':4.0,'fps':fps})
        patch_bvps = vhr.BVP.apply_filter(patch_bvps, BPfilter, params={'order':6,'minHz':0.65,'maxHz':4.0,'fps':fps})

        # Get the heatmaps
        image = sig_extractor.display_frame
        # Create directories if they don't exist
        os.makedirs(os.path.join(path, f"{method}_{camera}_estimated"), exist_ok=True)
        os.makedirs(os.path.join(path, f"{method}_{camera}_error"), exist_ok=True)

        for wind in range(len(patch_bvps)):
            # Get landmarks and visualizations
            ldmks, estimated_fig = vhr.plot.visualize_BVPs_heatmap(image, patch_bvps, patch_ids, wind, patch_shape, overlap, fps, minHz=0.65, maxHz=4)
            error_fig = vhr.plot.visualize_BPM_Errors_heatmap(image, ldmks, timesES, wind, timesGT_ECG, bpmGT_ECG, patch_shape, overlap, vmin=-20, vmax=20)
            
            # Construct the filenames
            estimated_filename = os.path.join(path, f"{method}_{camera}_estimated", f"{wind+1}.png")
            error_filename = os.path.join(path, f"{method}_{camera}_error", f"{wind+1}.png")
            
            # Save the figures using cv2.imwrite
            cv2.imwrite(estimated_filename, estimated_fig)
            cv2.imwrite(error_filename, error_fig)

        # Get BPM

        hol_bpmES = vhr.BPM.BVP_to_BPM_cuda(hol_bvps, fps)
        if method == "PCA":
            hol_bpmES = [x[0] for x in hol_bpmES]
        if method == "ICA":
            hol_bpmES = [x[0] for x in hol_bpmES]

        patch_bpmES = vhr.BPM.BVP_to_BPM_cuda(patch_bvps, fps)
        patch_median_bpmES, MAD = vhr.BPM.BPM_median(patch_bpmES)
        ma = vhr.extraction.MotionAnalysis(sig_extractor, wsize, fps)
        psd_bpmES = vhr.BPM.BPM_clustering(ma, patch_bvps, fps, wsize, movement_thrs=None, opt_factor=0.5)
            
        # Get the errors

        # Holistic errors
        if sigGT_ECG is not None:
            RMSE, MAE, MAX, PCC, CCC, SNR = getErrors(hol_bvps, fps, hol_bpmES, bpmGT_ECG, timesES, timesGT_ECG)
        elif sigGT_ABP is not None:
            RMSE, MAE, MAX, PCC, CCC, SNR = getErrors(hol_bvps, fps, hol_bpmES, bpmGT_ABP, timesES, timesGT_ABP)
        elif sigGT_CVP is not None:
            RMSE, MAE, MAX, PCC, CCC, SNR = getErrors(hol_bvps, fps, hol_bpmES, bpmGT_CVP, timesES, timesGT_CVP)

        # Update the DataFrame
        ERRORS = [RMSE, MAE, MAX, PCC, CCC, SNR]
        OUTPUT = [timesES, hol_bpmES, ERRORS]
        df.at[df[df['INSTANCE'] == int(instance)].index[0], 'HOL_' + method + '_' + camera] = OUTPUT

        # Patch errors
        # Medians
        if sigGT_ECG is not None:
            RMSE, MAE, MAX, PCC, CCC, SNR = getErrors(patch_bvps, fps, patch_median_bpmES, bpmGT_ECG, timesES, timesGT_ECG)
        elif sigGT_ABP is not None:
            RMSE, MAE, MAX, PCC, CCC, SNR = getErrors(patch_bvps, fps, patch_median_bpmES, bpmGT_ABP, timesES, timesGT_ABP)
        elif sigGT_CVP is not None:
            RMSE, MAE, MAX, PCC, CCC, SNR = getErrors(patch_bvps, fps, patch_median_bpmES, bpmGT_CVP, timesES, timesGT_CVP)

        # Update the DataFrame
        ERRORS = [RMSE, MAE, MAX, PCC, CCC, SNR]
        OUTPUT = [timesES, patch_median_bpmES, ERRORS]
        df.at[df[df['INSTANCE'] == int(instance)].index[0], 'PATCH_' + method + '_MED_' + camera] = OUTPUT

        # PSD Clustering
        if sigGT_ECG is not None:
            RMSE, MAE, MAX, PCC, CCC, SNR = getErrors(patch_bvps, fps, psd_bpmES, bpmGT_ECG, timesES, timesGT_ECG)
        elif sigGT_ABP is not None:
            RMSE, MAE, MAX, PCC, CCC, SNR = getErrors(patch_bvps, fps, psd_bpmES, bpmGT_ABP, timesES, timesGT_ABP)
        elif sigGT_CVP is not None:
            RMSE, MAE, MAX, PCC, CCC, SNR = getErrors(patch_bvps, fps, psd_bpmES, bpmGT_CVP, timesES, timesGT_CVP)

        # Update the DataFrame
        ERRORS = [RMSE, MAE, MAX, PCC, CCC, SNR]
        OUTPUT = [timesES, psd_bpmES, ERRORS]
        df.at[df[df['INSTANCE'] == int(instance)].index[0], 'PATCH_' + method + '_PSD_' + camera] = OUTPUT


## Run the function

In [ ]:
# Test the function
process_video(df,1,'K1')

In [ ]:
dataset_name = 'CVP'
window_info = [8,1]     # window size and stride
seconds = 0     # seconds of video to be processed (0 for all video)
skin_threshold = 30    # threshold for skin extraction
pixel_thresholds = [5,230] #low and high thresholding
frequency_bandpass = [0.65,4.0] # bandpass range
patch_info = [40,0] # size, pixel overlap

df = load_or_create_dataframe(results_path)
methods = ['CHROM', 'LGI', 'POS', 'PBV', 'GREEN', 'OMIT','ICA','PCA']

def process_cvp_video(df,instance,camera,methods,pixel_threshold=30,wsize=8,seconds=0):
    
    # Set the parameters
    dataset, sig_extractor = initialize_objects(df,instance,camera)
    
    # Read the ground truth
    read_ground_truth_data(df,instance,camera,dataset)

    # Holistic Processing
    df, sig_extractor = holistic_processing(df,instance,camera,sig_extractor,wsize,pixel_threshold)
    
        


In [ ]:
dataset_name = 'CVP'
window_info = [8,1]     # window size and stride
seconds = 0     # seconds of video to be processed (0 for all video)
skin_threshold = 30    # threshold for skin extraction
pixel_thresholds = [5,230] #low and high thresholding
frequency_bandpass = [0.65,4.0] # bandpass range
df = load_or_create_dataframe(results_path)
methods = ['CHROM', 'LGI', 'POS', 'PBV', 'GREEN', 'OMIT','ICA','PCA']
instance = 1
camera = "K1"

In [ ]:
df, dataset, sig_extractor = initialize_objects(df,instance,camera)

In [ ]:
df = read_ground_truth_data(df,instance,camera,dataset,window_info)

In [ ]:
df